In [1]:
import numpy as np
import pandas as pd
from datetime import datetime as dt

locations = "/home/tanglinh/Desktop/Musical Tweet Data/location.txt"
countries = "/home/tanglinh/Desktop/Musical Tweet Data/countryInfo.txt"
tweets = "/home/tanglinh/Desktop/Musical Tweet Data/tweet.txt"
tracks = "/home/tanglinh/Desktop/Musical Tweet Data/track.txt"
artists = "/home/tanglinh/Desktop/Musical Tweet Data/artists.txt"
mbartists = '/home/tanglinh/Downloads/mbdump/artist'

### PREPROCESSING DATA ###

#Create a data frame for tweet.txt and location.txt
tweets_df = pd.read_csv(tweets, header = None, sep='\t')
tweets_df.columns = ['file_id', 'tweet_id', 'user_id', 'artistId', 'trackId', 'datetime', 'weekday', 
                     'longitude', 'latitude']

locations_df = pd.read_csv(locations, header = 0, sep='\t')

tracks_df = pd.read_csv(tracks, header = 0, sep='\t').drop(0)

mbartists_df = pd.read_csv(mbartists, header = None, sep='\t', low_memory=False)
mbartists_df.columns = ['id', 'mbid', 'artists_name', 'sort_name', 'begin_date_year', 'begin_date_month', 'begin_date_day', 'end_date_year', 'end_date_month', 'end_date_day', 'artist_type', 'artist_area', 'artist_gender', 'comment', 'edits_pending', 'last_updated', 'ended', 'artist_born', 'artist_died']
mbartists_df = mbartists_df.drop(labels = ['id', 'sort_name', 'begin_date_year', 'begin_date_month', 'begin_date_day', 'end_date_year', 'end_date_month', 'end_date_day', 'comment', 'edits_pending', 'last_updated', 'ended', 'artist_born', 'artist_died'], axis=1)

artiststxt_df = pd.read_csv(artists, header = 0, sep='\t')
artiststxt_df = pd.merge(artiststxt_df, mbartists_df, left_on='artist_mbid', right_on='mbid')
artiststxt_df = artiststxt_df.drop(labels =['artist_name', 'artist_mbid', 'mbid'], axis=1)

#create the metadata frame: 
#joined_table_df = pd.merge(tweets_df, locations_df.drop_duplicates(['longitude', 'latitude']), on=['longitude', 'latitude'], how='right')
joined_table_df = pd.merge(tweets_df, tracks_df, left_on=['trackId', 'artistId'], right_on=['track_id', 'track_artistId'])
joined_table_df = pd.merge(joined_table_df, artiststxt_df, left_on='artistId', right_on='artist_id')

#extract months and timestamp info from datetime strings
joined_table_df['datetime'] = pd.to_datetime(joined_table_df['datetime'])
joined_table_df['tweet_month'] = joined_table_df['datetime'].dt.month
joined_table_df['tweet_year'] = joined_table_df['datetime'].dt.year

#cluster hours in the day: 5-12:Morning, 13-17: Afternoon, 18-21: Evening, 22-4: Night
hours = joined_table_df['datetime'].dt.hour
joined_table_df['tweet_time'] = pd.cut(hours-5+24 *(hours<5),bins=[0, 8, 12, 17, 25],labels=['Morning','Afternoon','Evening','Night'],right=False)

#cluster days in the week: Mon-Fri:weekday, Sat + Sun = Weekend
joined_table_df['tweet_day'] = pd.cut(joined_table_df['weekday'],bins=[0, 5, 7],labels=['weekday', 'weekend'],right=False)

#drop unneccesary columns
joined_table_df = joined_table_df.drop(labels = ['longitude', 'latitude', 'weekday', 'trackId', 'track_artistId', 'artistId', 'file_id', 'datetime'], axis=1)
joined_table_df.to_csv(index=False, path_or_buf="/home/tanglinh/Desktop/mydata.txt")
